In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# DO NOT FORGET TO CHECK IF THERE ARE MISSING PLAYER INFORMATION, SUCH AS MISSING POSITIONS. XPATH COULD BE DIFFERENT FOR A NUMBER OF PLAYERS.
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# required imports

import os
import pickle
import pandas as pd
import sys
import time
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import re
from datetime import date
from datetime import datetime
from dateutil import relativedelta
import numpy as np
import sqlite3 as sql3

In [ ]:
# var definitions

PATH = f"{os.path.abspath('')}\output"
FIX_PATH = f"{os.path.abspath('')}\output"

stat_dict = {}
counter = 0
master_dict = {}
refined_data = {}

In [ ]:
# concatenate WHOLE RAW DATABASE INTO ONE (there will be several files like "raw_data2023-08-19-2037.pkl")
# if you get a PermissionError, check file locations on your computer or var definitions

master_raw_data = []
for file in os.listdir(PATH):
    full_path = os.path.join(PATH,file) # find the raw outputs resulted by scraping and join
    with open(full_path,"rb") as fhand:
        raw_data = pickle.load(fhand)
    master_raw_data.extend(raw_data)
print(len(master_raw_data))
with open("master_raw_data.pkl","wb") as fhand:
    pickle.dump(master_raw_data,fhand)

In [ ]:
# data engineering and forming

master_data = []

with open("output/master_raw_data.pkl","rb") as fhand:
    full_raw_data = pickle.load(fhand) # load full raw data

for index in range(0,len(full_raw_data),2):

    for i in range(0,len(full_raw_data[index+1]),2):
        stat_dict[full_raw_data[index+1][i]] = full_raw_data[index+1][i+1]
        full_raw_data[index] = full_raw_data[index] | stat_dict
        stat_dict = {} 

    master_data.append(full_raw_data[index]) # turn lists into dictionaries (old form was {},[] per player) and merge the two dictionaries, making one full dictionary for one player


#### Execute here if you do not have a "master_data.pkl" file!
# df = pd.DataFrame(master_data)
# with open("output/master_data.pkl","wb") as fhand:
#   pickle.dump(df,fhand)        

for i in master_data:
    print(i)

In [ ]:
# Merge main df and the link df

import pickle
import pandas as pd

with open("output/master_data.pkl","rb") as fhand:
    main_df = pickle.load(fhand) # Load the main dataframe
    
with open("data/player_links.pkl","rb") as fhand:
    link_dict = pickle.load(fhand) # Load the link dictionary

print(link_dict)

print(len([x for x in link_dict]))

link_df = pd.DataFrame([x for x in link_dict.values()],index=[x for x in link_dict]) # Create a link dataframe from the link dictionary

frames = [main_df,link_df]

link_df = link_df.reset_index(names="playerlink") # since player links already exist in main_df, reset the link index on newly created link dataframe for merging

mdf = pd.merge(main_df,link_df,on="playerlink", how="left") # Merge the two dictionaries
mdf

In [ ]:
# Column removal ("price updated", columns with same headers)

mdf.drop(columns=["Price Updated"],inplace=True)
mdf.drop(columns=["DIVING","HANDLING","KICKING","REFLEXES","POSITIONING"],inplace=True)

mdf

In [ ]:
# Change the overall rating column name and upper column names

new_column_name = 'Overall Rating'
mdf.rename(columns={0: new_column_name}, inplace=True)

for x in mdf.columns:
    if x == x.upper() and not x == "ID" and not x == "DOB":
        renamedict = {}
        renamedict[x] = x.title()
        mdf.rename(columns=renamedict,inplace=True) # Detect full capitalized columns like "PACE", exclude "ID" and "DOB" and rename remaining
mdf

In [ ]:
# add height by cm and in

cm_list = []
in_list = []
str_ex = r'''([0-z]+'[0-z]+")'''


if mdf.columns[10] == "Height":

    for x in mdf.loc[:,"Height"]:
        stg = re.findall(".+?[0-9]+", x)[0]
        cm_list.append(stg) # Select cm values in the strings, eg. select aaa in "aaacm | bbbin"

    for x in mdf.loc[:, "Height"]:
        stg = re.findall(str_ex, x)[0]
        stg_without_backslash = stg.replace(r'\\', '')  # Remove backslashes from the captured value
        in_list.append(stg_without_backslash)

    n = mdf.columns[10]
    mdf.drop(n, axis = 1, inplace=True) # Drop the old height values
    mdf[n] = cm_list
    mdf.rename(columns={"Height": "Height (cm)"}, inplace=True) # Create the cm column
    mdf["Height (in)"] = in_list # Create the in column
else:
    pass

mdf

In [ ]:
# Fix Date of Birth/Age
converted_ages = []
counter = 0

def date_to_age(string):
    stg = re.findall("[0-9]+",string)
    final_str = f"{stg[2]}-{stg[1]}-{stg[0]}" # Date conversion from DD-MM-YYYY to YYYY-MM-DD

    today_date = datetime.strptime(str(date.today()), "%Y-%m-%d") 
    date_of_birth = datetime.strptime(final_str, "%Y-%m-%d") # convert strings to datetime objects

    delta = relativedelta.relativedelta(today_date,date_of_birth) # find the difference between two dates

    converted_ages.append(f"{delta.years} years old")

for index, value in enumerate(mdf.loc[:, "DOB"]):
    if not pd.isnull(value):
        date_to_age(value)
        age = converted_ages[counter]
        counter += 1
        mdf.loc[index, "Age"] = age # merge modified date of birth values to age

for index, value in enumerate(mdf.loc[:,"Age"]):
    str_ex = re.findall("[0-9]+", value)
    mdf.loc[index, "Age"] = str_ex[0] # remove "years old" strings

for x in mdf.loc[:,"Age"]:
    print(x)

In [ ]:
# if weight = 0 -> null

for index, value in enumerate(mdf.loc[:,"Weight"]):
    if value == "0":
        mdf.loc[index, "Weight"] = np.nan # find "0"s in weight values and convert them to np.nan

for x in mdf.loc[:, "Weight"]:
    if type(x) == float:
        print("nan found") # checker


In [ ]:
# if origin = "N\A" -> null

for index, value in enumerate(mdf.loc[:,"Origin"]):
    if value == "N\A":
        mdf.loc[index, "Origin"] = np.nan # find "N\A"s in origin values and convert them to np.nan

for x in mdf.loc[:, "Origin"]:
    if type(x) == float:
        print("nan found") # checker

In [ ]:
# Change data type of numbers and the same named columns

mdf.columns.values[39] = "Dribbling Main"
mdf.columns.values[44] = "Dribbling Alt"
mdf.columns    

counter = 0

for x in mdf.columns:
    for index, value in enumerate(mdf.loc[:, x]):
        counter += 1
        if isinstance(value, str) and value.isdigit(): #check if the "str" can be an "int"
            mdf.loc[index, x] = int(value)
        print(type(mdf.loc[index, x]))
    print(f"checked {x}")

print(counter)

In [ ]:
# Add a column "Date Added" (Date for testing: 2023-08-25)
# THIS BLOCK IS FOR TESTING ONLY. ACTUAL "Date Added" will be implemented in a different way.

test_date_obj = datetime.strptime("2023-08-25","%Y-%m-%d").date()
date_list = []

for i in mdf.loc[:,"Name"]:
    date_list.append(test_date_obj)
mdf["Date Added"] = date_list
mdf

In [ ]:
# Pickle the Master Dataframe

with open("output/master_dataframe.pkl","wb") as fhand:
    pickle.dump(mdf,fhand)

In [ ]:
# Unpickle existing & modified Master Dataframe

with open("output/master_dataframe.pkl","rb") as fhand:
    mdf = pickle.load(fhand)


In [ ]:
# SQLite Writer

conn = sql3.connect('database.db')

mdf.to_sql(name="player_database",con=conn)

In [ ]:
# Excel Writer

writer = pd.ExcelWriter('data/player_master_df.xlsx')# pylint: disable=abstract-class-instantiated
mdf.to_excel(writer, sheet_name="sheet1",index=False)
writer.close()

In [ ]:
# Check the indices of "Dribbling" columns

# Get boolean array indicating columns with the name "Dribbling"
dribbling_mask = mdf.columns == "Dribbling"

# Get index positions where "Dribbling" columns are located
dribbling_indices = [i for i, value in enumerate(dribbling_mask) if value]

print(dribbling_indices)